In [9]:
import psycopg2
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio

In [10]:
conn = psycopg2.connect(
    host="localhost",
    database="proyecto_5",
    user="postgres",
    password="admin"
)
cursor = conn.cursor()

In [11]:
df_cash_converters = pd.read_sql("SELECT precio FROM datos_cash_converters", conn)
df_mercado_libre = pd.read_sql("SELECT precio_eur AS precio FROM datos_mercado_libre", conn)
df_wallpop_limpios = pd.read_sql("SELECT precio FROM datos_wallpop_limpios", conn)
df_vinted_filtrados = pd.read_sql("SELECT total_item_price AS precio FROM datos_vinted_filtrados", conn)

df_cash_converters['Plataforma'] = 'Cash Converters'
df_mercado_libre['Plataforma'] = 'Mercado Libre'
df_wallpop_limpios['Plataforma'] = 'Wallapop'
df_vinted_filtrados['Plataforma'] = 'Vinted'

df_total = pd.concat([df_cash_converters, df_mercado_libre, df_wallpop_limpios, df_vinted_filtrados])

df_otros = df_total[df_total['Plataforma'] != 'Vinted']
df_vinted = df_total[df_total['Plataforma'] == 'Vinted']

trazas_otros = [
    go.Box(y=df_otros[df_otros['Plataforma'] == platform]['precio'].dropna(), name=platform)
    for platform in df_otros['Plataforma'].unique()
]

traza_vinted = go.Box(y=df_vinted['precio'].dropna(), name="Vinted")

fig_otros = go.Figure(data=trazas_otros)
fig_otros.update_layout(
    title="Distribución de Precios de Productos de Segunda Mano por Plataforma (sin Vinted)",
    xaxis_title="Plataforma",
    yaxis_title="Precio de Productos de Segunda Mano"
)

fig_vinted = go.Figure(data=[traza_vinted])
fig_vinted.update_layout(
    title="Distribución de Precios de Productos de Segunda Mano en Vinted",
    xaxis_title="Plataforma",
    yaxis_title="Precio de Productos de Segunda Mano"
)

fig_otros.show()
fig_vinted.show()


C:\Users\almaz\AppData\Local\Temp\ipykernel_13336\2520694407.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\almaz\AppData\Local\Temp\ipykernel_13336\2520694407.py:3: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\almaz\AppData\Local\Temp\ipykernel_13336\2520694407.py:4: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\almaz\AppData\Local\Temp\ipykernel_13336\2520694407.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 conn

## Conclusiones

### Conclusiones sobre los Datos de Precios en Vinted

Al analizar los datos de precios en las distintas plataformas, podemos observar varias particularidades en los resultados provenientes de **Vinted**:

1. **Precios Inflados para Negociación**: En Vinted, la distribución de precios muestra valores extremadamente altos, con un precio máximo de **31,500** €. Sin embargo, al observar detenidamente, parece que la mayoría de estos precios elevados no representan el valor real de una PlayStation 5, sino que se utilizan para abrir una negociación. Es común que en plataformas como Wallapop y Vinted, los vendedores coloquen precios muy altos o muy bajos (como 0 €) con el propósito de invitar a los compradores a negociar.

2. **Productos Relacionados, No Consolas**: En el caso de Vinted, la mayoría de los artículos con nombres que contienen "PlayStation 5" no son consolas propiamente dichas, sino productos relacionados. Esto podría incluir accesorios, juegos, o incluso cajas vacías de la consola. De hecho, solo hay unos pocos (alrededor de 5) artículos relacionados con una PlayStation 5, lo que sugiere que los datos de esta plataforma pueden no representar de manera precisa los precios de consolas de segunda mano en comparación con otras plataformas.

3. **Dificultad para Comparar**: Dado que estos precios no reflejan directamente el valor real de la consola y que muchos de los productos son accesorios o artículos relacionados, los datos de Vinted distorsionan la comparación con otras plataformas. Para un análisis más representativo, sería conveniente filtrar los productos específicos de PlayStation 5 en lugar de incluir todos los artículos etiquetados bajo ese nombre.

4. **Prácticas de Mercado en Vinted y Wallapop**: Es interesante notar cómo los precios en plataformas de segunda mano pueden estar manipulados para crear espacio para negociaciones. Esto es especialmente común en plataformas particular-particular como Wallapop y Vinted, donde las expectativas de negociación son altas, a diferencia de plataformas de venta directa como Mercado Libre o Cash Converters.

En conclusión, los datos de precios en Vinted no parecen representar de manera precisa el valor de mercado de las consolas PlayStation 5 de segunda mano debido a la mezcla de artículos relacionados y la estrategia de precios inflados para negociar. Para obtener conclusiones más realistas, sería necesario un filtrado más específico de productos y precios.

### Conclusiones sobre la Distribución de Precios en Cash Converters, Mercado Libre y Wallapop

Al analizar la distribución de precios de productos de segunda mano en las plataformas **Cash Converters**, **Mercado Libre** y **Wallapop** (sin incluir Vinted), se observan varias diferencias clave:

1. **Mercado Libre tiene la mayor variabilidad de precios**:
   - La caja de Mercado Libre es mucho más grande que las de las otras dos plataformas, lo cual indica una mayor dispersión de precios.
   - Este rango amplio sugiere que en Mercado Libre hay tanto productos de bajo costo como de precios más altos, posiblemente debido a la presencia de diversos tipos de artículos relacionados con la PlayStation 5 (como accesorios, juegos, y consolas en diferentes estados de conservación).
   - El precio medio (mediana) se encuentra en un punto intermedio dentro de esta dispersión, lo cual sugiere que, aunque hay productos de menor valor, también existen opciones de precio considerablemente elevado.

2. **Cash Converters presenta una distribución de precios más estable y limitada**:
   - La caja de Cash Converters es pequeña, lo que indica que la mayoría de los precios están agrupados en un rango estrecho, alrededor de los 400-500 €. 
   - Esto sugiere una política de precios más uniforme, probablemente debido a controles de precio que implementa la propia plataforma al comprar y revender productos de segunda mano.
   - Cash Converters tiene pocos valores atípicos, lo que demuestra una menor variabilidad en el precio de los productos que ofrece en comparación con Mercado Libre.

3. **Wallapop tiene precios generalmente bajos y una dispersión reducida**:
   - En Wallapop, los precios están mayormente agrupados en el rango inferior, lo que indica una tendencia a precios más accesibles.
   - La caja es muy pequeña, lo cual señala poca variabilidad en los precios. Esto puede deberse a que en Wallapop predominan los vendedores particulares, que generalmente fijan precios menores para atraer compradores en una plataforma altamente competitiva.
   - Además, se observa un valor atípico bajo (outlier), probablemente algún accesorio de bajo costo o incluso un producto listado a 0 € para abrir negociación.

4. **Diferencias en estrategias de precios y tipo de vendedor**:
   - En Mercado Libre, donde suelen haber tanto vendedores individuales como tiendas pequeñas, los precios muestran más variabilidad. Esto puede estar influenciado por una mezcla de artículos nuevos y de segunda mano, así como distintos tipos de vendedores que intentan optimizar sus ganancias.
   - En Cash Converters, que es una tienda de segunda mano con precios regulados, los precios son más estables y menos competitivos en cuanto a oferta.
   - Wallapop, que es una plataforma peer-to-peer, tiene precios más bajos debido a la alta competencia entre vendedores particulares que intentan vender rápidamente.

### Conclusión

La gráfica sugiere que **Cash Converters** es la plataforma con los precios más uniformes, lo que podría reflejar una mayor fiabilidad en cuanto a la calidad y precio de los productos. **Mercado Libre** presenta la mayor variabilidad, probablemente debido a la diversidad de artículos y tipos de vendedores, mientras que **Wallapop** ofrece en promedio los precios más bajos, posiblemente debido a la dinámica de venta entre particulares y la alta competencia en precios.

Esta comparación sugiere que, dependiendo del presupuesto y de la necesidad de seguridad en la compra, los usuarios podrían preferir una plataforma sobre otra. Los compradores con un presupuesto ajustado probablemente optarán por Wallapop, mientras que aquellos que buscan una compra más confiable y están dispuestos a pagar un poco más podrían preferir Cash Converters.

> Hay que tener en cuenta que la plataforma `Mercado Libre` no es una plataforma que opere a nivel nacional (España), ha habido un cambio de moneda por el camino a la hora de realizar el análisis
